# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,49.44,89,100,6.02,CL,1723492389
1,1,ducheng,23.2373,111.5317,78.01,94,100,1.28,CN,1723492389
2,2,mariental,-24.6333,17.9667,64.00,15,1,10.74,NaN,1723492389
3,3,port-aux-francais,-49.3500,70.2167,37.58,83,56,29.89,TF,1723492389
4,4,qulsary,46.9531,54.0198,73.65,55,31,15.30,KZ,1723492389


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City",
    frame_width=800,
    frame_height=500
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_temp = city_data_df[(city_data_df['Max Temp']<90) & (city_data_df['Max Temp']>55)]
filtered_wind = filtered_temp[filtered_temp['Wind Speed']>4.0]
filtered_city = filtered_wind[filtered_wind['Cloudiness']<40]

# Drop any rows with null values
filtered_df = filtered_city.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,qulsary,46.9531,54.0198,73.65,55,31,15.30,KZ,1723492389
5,5,hovd,48.0056,91.6419,63.30,59,23,5.19,MN,1723492389
9,9,carnarvon,-24.8667,113.6333,63.46,76,0,9.89,AU,1723492389
12,12,praia da vitoria,38.7333,-27.0667,80.85,61,20,8.39,PT,1723492390
23,23,guerrero negro,27.9769,-114.0611,83.84,50,0,12.28,MX,1723492390


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df.copy()
hotel_df = hotel_df.drop(['Max Temp','Cloudiness','Wind Speed'],axis=1)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Humidity,Country,Date,Hotel Name
4,4,qulsary,46.9531,54.0198,55,KZ,1723492389,
5,5,hovd,48.0056,91.6419,59,MN,1723492389,
9,9,carnarvon,-24.8667,113.6333,76,AU,1723492389,
12,12,praia da vitoria,38.7333,-27.0667,61,PT,1723492390,
23,23,guerrero negro,27.9769,-114.0611,50,MX,1723492390,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qulsary - nearest hotel: Камилла
hovd - nearest hotel: Khovd hotel
carnarvon - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
guerrero negro - nearest hotel: Plaza sal paraiso
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
daru - nearest hotel: Kuki
bokovskaya - nearest hotel: No hotel found
querqueville - nearest hotel: Shelder Hôtel
alofi - nearest hotel: Matavai Resort
west island - nearest hotel: Cocos Village Bungalows
lompoc - nearest hotel: Red Roof Inn Lompoc
nili - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
kas - nearest hotel: KEKOVA OTEL
yueyang - nearest hotel: 南湖宾馆
guatrache - nearest hotel: No hotel found
veinticinco de mayo - nearest hotel: Jorly
batesville - nearest hotel: Hampton Inn Batesville
mana - nearest hotel: Le Samana
road town - nearest hotel: No hotel found
port arthur - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
saipan - nearest hotel: Chalan K

,City_ID,City,Lat,Lng,Humidity,Country,Date,Hotel Name
4,4,qulsary,46.9531,54.0198,55,KZ,1723492389,Камилла
5,5,hovd,48.0056,91.6419,59,MN,1723492389,Khovd hotel
9,9,carnarvon,-24.8667,113.6333,76,AU,1723492389,No hotel found
12,12,praia da vitoria,38.7333,-27.0667,61,PT,1723492390,Salles
23,23,guerrero negro,27.9769,-114.0611,50,MX,1723492390,Plaza sal paraiso
...,...,...,...,...,...,...,...,...
571,571,villarrica,-25.7500,-56.4333,36,PY,1723492219,No hotel found
578,578,fontanelas,38.8472,-9.4372,80,PT,1723492427,Azen Cool House
592,592,bubaque,11.2833,-15.8333,87,GW,1723492428,Chez Julio
594,594,piacabucu,-10.4056,-36.4344,82,BR,1723492428,O Leão


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 0.5,
    color = "City",
    frame_width=600,
    frame_height=350,
    hover_cols=["City", "Humidity", "Hotel Name", "Country", "Lng", "Lat"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)